In [8]:
#!pip install -U openai-whisper

In [41]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
import datasets
from datasets import Audio, load_dataset
import soundfile
import librosa
import torch
from evaluate import load
from IPython.display import Audio, display

In [22]:
# Load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

In [46]:
# Load dummy dataset and read audio files
ds = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="validation", streaming=True)
#ds = ds.cast_column("audio", Audio(rate=16000))
input_speech = next(iter(ds))
input_speech
#input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

Reading metadata...: 16354it [00:01, 12073.23it/s]


ModuleNotFoundError: No module named 'librosa.core'

In [ ]:
!pip uninstall librosa

In [ ]:
# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

In [ ]:
dsTest = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test", streaming=True)
dsTest = dsTest.cast_column("audio", Audio(sampling_rate=16_000))
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [ ]:
def map_to_pred(batch):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['text'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch


In [ ]:
result = dsTest.map(map_to_pred)
dsTest

In [ ]:
wer=load("wer")
wer

In [ ]:
print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))

In [ ]:
# Sample Speech audio file
sample_speech1 = display(Audio('Conference.wav', autoplay=True))

In [ ]:
input_features1 = processor(sample_speech1).input_features